In [1]:
import os
import glob
import json
import pandas as pd

In [2]:
# Good one!!

import instaloader
import pandas as pd

# Creating an instance of the Instaloader class
bot = instaloader.Instaloader()
user = "ca.seguin"
# Load a session created from a user session file
try:
    bot.load_session_from_file(user)
except FileNotFoundError:
    # If the session file doesn't exist, perform a login and save the session
    bot.context.log("Session file does not exist yet - Logging in.")
    bot.interactive_login(user)
    bot.save_session_to_file()

# Load the profile
profile = instaloader.Profile.from_username(bot.context, user)

# Get followers and followees
followers = set(f.username for f in profile.get_followers())
followees = set(f.username for f in profile.get_followees())

# Find followees that are not followers
non_followers = followees - followers

# Create a DataFrame
data = {'Followees_Not_Followers': list(non_followers)}
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

Loaded session from C:\Users\Charles_tour\AppData\Local\Instaloader\session-ca.seguin.
   Followees_Not_Followers
0            igloofest_mtl
1                 zimblake
2                 vinny_t_
3                 cigaleqc
4               mariona_cr
5          sunshinevillage
6            remymetailler
7                  osheaga
8      parc.des.appalaches
9             drdisrespect
10            rickandmorty
11             lefestifbsp
12                ilesoniq
13           electricpants
14            festivalnorr
15          ultra_pcbuilds
16          portugaltheman
17    catherinedeslauriers
18          modernsciencex
19               landmvrks
20           martyschwartz
21           festivaleteqc
22            timthetatman
23         ronieblackheart
24            boskvelocafe
25               lambofgod


In [ ]:
import instaloader
 
# Creating an instance of the Instaloader class
bot = instaloader.Instaloader()

def getBasicInfo():
    # profileid = input('Enter the userid of the profile\n')
    profileid = "ca.seguin"
    # Loading the profile from an Instagram handle
    profile = instaloader.Profile.from_username(bot.context, profileid)
    print("Username: ", profile.username)
    print("User ID: ", profile.userid)
    print("Number of Posts: ", profile.mediacount)
    print("Followers Count: ", profile.followers)
    print("Following Count: ", profile.followees)
    print("Bio: ", profile.biography)
    print("External URL: ", profile.external_url) 
    print(profile.get_followers())  
    print('\n')
    
getBasicInfo()

In [3]:
cwd = os.getcwd()
print(cwd)

c:\Users\Charles_tour\Documents\GitHub\Instagram_followers


In [8]:
# Lister les folder dispo dans le répoertoire
cwd = os.getcwd()
print(cwd)

files =  glob.glob(os.path.join(cwd, '*/'))

for i in files:
    print(i)

# ['/home/charles/Documents/Followers/mpsguin_20230422/', '/home/charles/Documents/Followers/ca.seguin_20230412/', '/home/charles/Documents/Followers/ca.seguin_20230424/', '/home/charles/Documents/Followers/ca.seguin_20230421/']
# ca_seguin_folders = [f for f in files if f.startswith('/home/charles/Documents/Followers/ca.seguin')]
# print(ca_seguin_folders)

c:\Users\Charles_tour\Documents\GitHub\Instagram_followers
c:\Users\Charles_tour\Documents\GitHub\Instagram_followers\ca.seguin_20230617\
c:\Users\Charles_tour\Documents\GitHub\Instagram_followers\ca.seguin_20230731\


In [3]:
# Compte qui se sont désabonnés
old_follower_folder = '/home/charles/Documents/Followers/ca.seguin_20230617/'
new_follower_folder = '/home/charles/Documents/Followers/ca.seguin_20230731/'

folder = 'followers_and_following/'

with open(old_follower_folder + folder + 'followers_1.json','r') as f:
    old_follower = json.loads(f.read())

with open(new_follower_folder + folder + 'followers_1.json','r') as f:
    new_follower = json.loads(f.read())

old_follower = pd.json_normalize(old_follower, record_path =['string_list_data'])
new_follower = pd.json_normalize(new_follower, record_path =['string_list_data'])

# Merge
merged_df = pd.merge(old_follower, new_follower, on='value', how='outer', indicator=True)

merge_cl = merged_df.loc[merged_df['_merge']=='left_only']
merge_cl = merge_cl[['value']].rename(columns={'value':'unfollowers'}).sort_values('unfollowers').reset_index(drop=True)

print('Liste des comptes qui se sont désabonnées')
merge_cl


Liste des comptes qui se sont désabonnées


,unfollowers
0,___danikacamp
1,__marieeve
2,_abeaudry
3,_alexanejoly_
4,_emlam
...,...
578,xavier_luneau_gagne31
579,yan.mso
580,yan_and_rik
581,yanlariviere_


In [4]:
# Comptes non-abonnés en retour
import os
import glob
import json
import pandas as pd

folder_path = '/home/charles/Documents/Followers/'

files =  glob.glob(os.path.join(folder_path, '*/'))
last_file = max(files, key=os.path.getmtime)
print('File use : ', last_file)

# path = '/home/charles/Documents/Followers/mpsguin_20230422/followers_and_following/'
folder = 'followers_and_following/'

with open(last_file + folder + 'following.json') as f:
    following = json.load(f)

data = [(string_list_data['href'], string_list_data['value'], string_list_data['timestamp'])
        for relationship in following['relationships_following']
        for string_list_data in relationship['string_list_data']]
following = pd.DataFrame(data, columns=['href', 'value', 'timestamp'])


# Follower
with open(last_file + folder + 'followers_1.json','r') as f:
    follower = json.loads(f.read())
follower = pd.json_normalize(follower, record_path =['string_list_data'])


# Merge
merged_df = pd.merge(following, follower, on='value', how='outer', indicator=True)

merge_cl = merged_df.loc[merged_df['_merge']=='left_only']
merge_cl = merge_cl[['value']].rename(columns={'value':'non_followers'}).sort_values('non_followers').reset_index(drop=True)

# to Excel
final_file_name = 'non_follower.xlsx'
merge_cl.to_excel(folder_path + final_file_name)

merge_cl

File use :  /home/charles/Documents/Followers/ca.seguin_20230731/


,non_followers
0,boskvelocafe
1,cigaleqc
2,drdisrespect
3,electricpants
4,festivaleteqc
5,festivalnorr
6,igloofest_mtl
7,ilesoniq
8,lambofgod
9,landmvrks
